In [1]:
from bula_process.models import Mistral, Rag, Llama3
import json 

model = Llama3()
rag = Rag()

system_prompt = """
Always answer the question in Brazilian Portuguese, being as simple and concise as possible.
Return ONLY the answer.
Example:
Quais efeitos colaterais podem ocorrer com a administração prolongada do Mud Oral?
Answer:
Hipertensão arterial e taquicardia.
"""

llm_judge_prompt_1 = """
Considering the question '{question}' and the following answer '{generated_answer}', a similar alternative answer is: 
"""

llm_judge_prompt_2 = """
Considering the question '{question}' and the following answer '{generated_answer}', is '{option}' a similar alternative answer?
"""

llm_judge_prompt_3 = """
Considering the question '{question}' and the following answer '{generated_answer}', select the option with the most similar alternative answer: 
{options}
5) None of the options is similar to the text.
Option:
"""

path = 'results/bula_all.jsonl'
list_json = []
with open(path, 'r') as file:
    for line in file:
        list_json.append(json.loads(line))

def make_question_rag(id, use_rag):
    data = list_json[id]
    question_prompt = ''

    
    if use_rag:
        context = rag.retrieve(data['query'])
        question_prompt = f"[INST]Answer the question: [/INST] Context: {context} \n {data['query']} \nAnswer: "
    else:
        question_prompt = f"[INST]Answer the question: [/INST] \n {data['query']} \nAnswer: "
   
    answer_model = model.inference(system_prompt + question_prompt)   


    list_prompt_1 = []
    list_prompt_2 = []
    list_prompt_3 = []
    options = '\n'.join([f"{index}) {content}" for index, content in enumerate(data['choices'])])
    for idx, choice in enumerate(data['choices']):
        prompt_1 = llm_judge_prompt_1.format(question=data['query'], generated_answer=answer_model)
        perplexity_question_1 = model.perplexity(prompt_1, choice)
        prompt_2 = llm_judge_prompt_2.format(question=data['query'], generated_answer=answer_model, option=choice)
        perplexity_question_2 = model.perplexity(prompt_2, "Yes")
        prompt_3 = llm_judge_prompt_3.format(question=data['query'], generated_answer=answer_model, options=options)
        perplexity_question_3 = model.perplexity(prompt_3, f"{idx}) {choice}")
       

        if idx == int(data['gold']):
            gold_choice = choice
            list_prompt_1.append(f"[{choice}]: {perplexity_question_1:.2f}")
            list_prompt_2.append(f"[{choice}]: {perplexity_question_2:.2f}")
            list_prompt_3.append(f"[{choice}]: {perplexity_question_3:.2f}")
        else:
            list_prompt_1.append(f"{choice}: {perplexity_question_1:.2f}")
            list_prompt_2.append(f"{choice}: {perplexity_question_2:.2f}")
            list_prompt_3.append(f"{choice}: {perplexity_question_3:.2f}")

    print(data['query'])
    print(f"Resposta gerada: {answer_model}")


    for ppl in list_prompt_1:
        print(ppl)
    print('\n')
    for ppl in list_prompt_2:
        print(ppl)
    print('\n')
    for ppl in list_prompt_3:
        print(ppl)
        



/home/navarro/miniconda3/envs/python310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/navarro/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.64it/s]
/home/navarro/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/navarro/.local/lib/python3.10/site-packages/huggingface_hub

In [2]:
make_question_rag(109, True)

Quais são os sintomas de efeitos colaterais que podem ocorrer com o uso de Neo Mistatin creme?
Resposta gerada: Irritação e sensibilidade, incluindo sensação de queimação e prurido.
dores de dente ou problemas de visão: 37.40
perda de peso ou dor de cabeça: 25.27
falta de ar ou dor no peito: 22.70
[irritação e sensibilidade, incluindo sensação de queimação e coceira]: 2.62


dores de dente ou problemas de visão: 8.57
perda de peso ou dor de cabeça: 8.24
falta de ar ou dor no peito: 6.66
[irritação e sensibilidade, incluindo sensação de queimação e coceira]: 2.80


dores de dente ou problemas de visão: 1.65
perda de peso ou dor de cabeça: 1.60
falta de ar ou dor no peito: 1.63
[irritação e sensibilidade, incluindo sensação de queimação e coceira]: 1.42


In [3]:
make_question_rag(109, False)

Quais são os sintomas de efeitos colaterais que podem ocorrer com o uso de Neo Mistatin creme?
Resposta gerada: Prurido, queimor e dermatite alérgica.
dores de dente ou problemas de visão: 27.94
perda de peso ou dor de cabeça: 26.29
falta de ar ou dor no peito: 18.08
[irritação e sensibilidade, incluindo sensação de queimação e coceira]: 10.51


dores de dente ou problemas de visão: 12.92
perda de peso ou dor de cabeça: 10.61
falta de ar ou dor no peito: 10.30
[irritação e sensibilidade, incluindo sensação de queimação e coceira]: 2.65


dores de dente ou problemas de visão: 1.72
perda de peso ou dor de cabeça: 1.58
falta de ar ou dor no peito: 1.66
[irritação e sensibilidade, incluindo sensação de queimação e coceira]: 1.15


In [4]:
make_question_rag(95, True)

Qual é a reação muito comum (>1/10) que pode ocorrer com o uso de Oxaliplatina e é relacionada à função hematológica?
Resposta gerada: Leucopenia.
Fosfatase alcalina anormal: 16.51
Dor abdominal: 97.77
Constipação: 127.48
[Anemia]: 3.19


Fosfatase alcalina anormal: 8.47
Dor abdominal: 10.76
Constipação: 12.23
[Anemia]: 7.24


Fosfatase alcalina anormal: 1.49
Dor abdominal: 3.03
Constipação: 2.49
[Anemia]: 2.37


In [5]:
make_question_rag(95, False)

Qual é a reação muito comum (>1/10) que pode ocorrer com o uso de Oxaliplatina e é relacionada à função hematológica?
Resposta gerada: Anemia.
Fosfatase alcalina anormal: 24.59
Dor abdominal: 182.43
Constipação: 111.38
[Anemia]: 3.05


Fosfatase alcalina anormal: 7.63
Dor abdominal: 10.62
Constipação: 10.04
[Anemia]: 4.82


Fosfatase alcalina anormal: 1.50
Dor abdominal: 3.03
Constipação: 2.36
[Anemia]: 2.14


In [6]:
make_question_rag(3, True)

Quais são os efeitos colaterais comuns que podem ocorrer com o uso de Cefuroxima sódica?
Resposta gerada: Neutropenia, eosinofilia, aumento transitório das enzimas hepáticas, reações no local da injeção.
Elevações na creatinina sérica, elevações no nitrogênio (ureia) sanguíneo, redução da velocidade de eliminação de creatinina pela urina: 8.07
[Diminuição das células de defesa do organismo (neutrófilos), aumento da quantidade de eosinófilos (células brancas) no sangue; aumento transitório das enzimas do fígado]: 3.42
Crescimento de cândida (tipo de fungo); redução do número de células de coagulação (plaquetas); febre medicamentosa: 15.77
Número inadequado de glóbulos vermelhos circulantes; processo inflamatório dos rins, reação alérgica severa: 9.29


Elevações na creatinina sérica, elevações no nitrogênio (ureia) sanguíneo, redução da velocidade de eliminação de creatinina pela urina: 5.00
[Diminuição das células de defesa do organismo (neutrófilos), aumento da quantidade de eosinófil

In [7]:
make_question_rag(3, False)

Quais são os efeitos colaterais comuns que podem ocorrer com o uso de Cefuroxima sódica?
Resposta gerada: Diarreia, vômitos e dor de cabeça.
Elevações na creatinina sérica, elevações no nitrogênio (ureia) sanguíneo, redução da velocidade de eliminação de creatinina pela urina: 8.91
[Diminuição das células de defesa do organismo (neutrófilos), aumento da quantidade de eosinófilos (células brancas) no sangue; aumento transitório das enzimas do fígado]: 6.88
Crescimento de cândida (tipo de fungo); redução do número de células de coagulação (plaquetas); febre medicamentosa: 18.82
Número inadequado de glóbulos vermelhos circulantes; processo inflamatório dos rins, reação alérgica severa: 11.46


Elevações na creatinina sérica, elevações no nitrogênio (ureia) sanguíneo, redução da velocidade de eliminação de creatinina pela urina: 4.10
[Diminuição das células de defesa do organismo (neutrófilos), aumento da quantidade de eosinófilos (células brancas) no sangue; aumento transitório das enzima

In [8]:
make_question_rag(5, True)

Qual é o efeito colateral comum do PEYONA que ocorre entre 1% e 10% dos pacientes que utilizam este medicamento?
Resposta gerada: Flebite e alterações inflamatórias locais.
Infecção do sangue: 30.81
Doenças cardíacas: 5.16
[Reações inflamatórias locais no lugar da perfusão]: 18.38
Surdez: 683.20


Infecção do sangue: 5.90
Doenças cardíacas: 8.83
[Reações inflamatórias locais no lugar da perfusão]: 3.84
Surdez: 8.62


Infecção do sangue: 1.90
Doenças cardíacas: 1.74
[Reações inflamatórias locais no lugar da perfusão]: 1.30
Surdez: 2.91


In [9]:
make_question_rag(5, False)

Qual é o efeito colateral comum do PEYONA que ocorre entre 1% e 10% dos pacientes que utilizam este medicamento?
Resposta gerada: Dor de cabeça.
Infecção do sangue: 39.31
Doenças cardíacas: 5.37
[Reações inflamatórias locais no lugar da perfusão]: 27.84
Surdez: 396.77


Infecção do sangue: 6.56
Doenças cardíacas: 7.10
[Reações inflamatórias locais no lugar da perfusão]: 6.81
Surdez: 6.81


Infecção do sangue: 1.94
Doenças cardíacas: 1.67
[Reações inflamatórias locais no lugar da perfusão]: 1.31
Surdez: 2.71


In [10]:
make_question_rag(8, True)

Quais são os efeitos colaterais mais comuns que podem ocorrer com o uso de Hidroquinona?
Resposta gerada: Eritemia, sensação leve de queimação e hipersensibilidade ocasional.
Hiperpigmentação de coloração marrom ou preto-azulada, sensação de dor e febre: 11.94
Manchas marrons reversíveis nas unhas, dor de cabeça e fadiga: 8.68
[Vermelhidão, coceira, descamação, inflamação excessiva, vesículas (bolhas) e sensação leve de queimação]: 8.06
Eczema, urticária e reações alérgicas graves: 4.36


Hiperpigmentação de coloração marrom ou preto-azulada, sensação de dor e febre: 5.36
Manchas marrons reversíveis nas unhas, dor de cabeça e fadiga: 5.17
[Vermelhidão, coceira, descamação, inflamação excessiva, vesículas (bolhas) e sensação leve de queimação]: 6.03
Eczema, urticária e reações alérgicas graves: 4.11


Hiperpigmentação de coloração marrom ou preto-azulada, sensação de dor e febre: 1.21
Manchas marrons reversíveis nas unhas, dor de cabeça e fadiga: 1.21
[Vermelhidão, coceira, descamação, 